In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['HF']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('heart failure', 3980),
 ('high fat', 1195),
 ('high frequency', 236),
 ('high fat diet', 210),
 ('hair follicle', 180),
 ('halofuginone', 88),
 ('high fructose', 36),
 ('human fibroblasts', 31),
 ('hydroxyflutamide', 30),
 ('hippocampal formation', 28),
 ('hemorrhagic fever', 24),
 ('hydrogen fluoride', 21),
 ('hip fracture', 20),
 ('hemofiltration', 20),
 ('hepatic fibrosis', 17),
 ('halofantrine', 15),
 ('high fat diet group', 15),
 ('high flow', 14),
 ('high frequency power', 14),
 ('hfd', 12),
 ('hyperforin', 12),
 ('hydrofluoric', 12),
 ('holstein friesian', 11),
 ('high fat low carbohydrate', 11),
 ('hollow fiber', 10),
 ('hf', 9),
 ('haemofiltration', 9),
 ('high frequency band', 9),
 ('human foreskin fibroblasts', 9),
 ('health facilities', 8),
 ('hageman factor', 8),
 ('housing first', 7),
 ('hexane fraction', 7),
 ('hydroxyfasudil', 7),
 ('high saturated fat', 7),
 ('fat high cholesterol', 7),
 ('hypofractionated', 6),
 ('hyperfibrinolysis', 6),
 ('hyperfine', 6),
 ('high 

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-10-06 01:37:54] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'fat high cholesterol': 'MESH:D059305',
  'fat high sugar': 'MESH:D059305',
  'haemofiltration': 'MESH:D006440',
  'hageman factor': 'HGNC:3530',
  'hair follicle': 'MESH:D018859',
  'halofantrine': 'CHEBI:CHEBI:5612',
  'halofuginone': 'MESH:C010176',
  'hartree fock': 'ungrounded',
  'health facilities': 'MESH:D006268',
  'heart failure': 'MESH:D006333',
  'hemofiltration': 'MESH:D006440',
  'hemorrhagic fever': 'NCIT:C36169',
  'hepatic fibrosis': 'NCIT:C97071',
  'hereditary ferritinopathy': 'MESH:C548080',
  'hexane fraction': 'ungrounded',
  'hf': 'ungrounded',
  'hf diet': 'MESH:D059305',
  'hf diet treated group': 'MESH:D059305',
  'hf rats treated with vehicle': 'ungrounded',
  'hfd': 'MESH:D059305',
  'high fat': 'MESH:D059305',
  'high fat and high sucrose': 'MESH:D059305',
  'high fat control': 'MESH:D059305',
  'high fat diet': 'MESH:D059305',
  'high fat diet group': 'high_frequency',
  'high fat feeding': 'MESH:D059305',
  'high fat group': 'CHEBI:CHEBI:24433',
  'high

In [6]:
grounding_map, names, pos_labels = [{'fat high cholesterol': 'MESH:D059305',
  'fat high sugar': 'MESH:D059305',
  'haemofiltration': 'MESH:D006440',
  'hageman factor': 'HGNC:3530',
  'hair follicle': 'MESH:D018859',
  'halofantrine': 'CHEBI:CHEBI:5612',
  'halofuginone': 'MESH:C010176',
  'hartree fock': 'ungrounded',
  'health facilities': 'MESH:D006268',
  'heart failure': 'MESH:D006333',
  'hemofiltration': 'MESH:D006440',
  'hemorrhagic fever': 'NCIT:C36169',
  'hepatic fibrosis': 'NCIT:C97071',
  'hereditary ferritinopathy': 'MESH:C548080',
  'hexane fraction': 'ungrounded',
  'hf': 'ungrounded',
  'hf diet': 'MESH:D059305',
  'hf diet treated group': 'MESH:D059305',
  'hf rats treated with vehicle': 'ungrounded',
  'hfd': 'MESH:D059305',
  'high fat': 'MESH:D059305',
  'high fat and high sucrose': 'MESH:D059305',
  'high fat control': 'MESH:D059305',
  'high fat diet': 'MESH:D059305',
  'high fat diet group': 'MESH:D059305',
  'high fat feeding': 'MESH:D059305',
  'high fat group': 'MESH:D059305',
  'high fat low carbohydrate': 'MESH:D059305',
  'high fat overfeeding': 'MESH:D059305',
  'high fatty': 'MESH:D059305',
  'high fiber': 'ungrounded',
  'high field': 'ungrounded',
  'high flow': 'ungrounded',
  'high flux': 'ungrounded',
  'high frequency': 'high_frequency',
  'high frequency band': 'high_frequency',
  'high frequency component': 'high_frequency',
  'high frequency power': 'high_frequency',
  'high frequency range': 'high_frequency',
  'high fructose': 'high_fructose',
  'high in fat': 'MESH:D059305',
  'high prebiotic fiber': 'ungrounded',
  'high saturated fat': 'MESH:D059305',
  'hinokiflavone': 'CHEBI:CHEBI:5721',
  'hip flexion': 'ungrounded',
  'hip fracture': 'MESH:D006620',
  'hippocampal formation': 'MESH:D006624',
  'histone fold': 'ungrounded',
  'hollow fiber': 'ungrounded',
  'holstein friesian': 'ungrounded',
  'hot flash': 'ungrounded',
  'hot flushes': 'ungrounded',
  'housing first': 'ungrounded',
  'human factor': 'ungrounded',
  'human fibrinogen': 'FPLX:Fibrinogen',
  'human fibroblasts': 'MESH:D005347',
  'human foreskin fibroblasts': 'MESH:D005347',
  'human skin fibroblasts': 'MESH:D005347',
  'hydatid fluid': 'ungrounded',
  'hydrofluoric': 'CHEBI:CHEBI:29228',
  'hydrogen fluoride': 'CHEBI:CHEBI:29228',
  'hydrops fetalis': 'MESH:D015160',
  'hydroxyfasudil': 'MESH:C431085',
  'hydroxyflavone': 'CHEBI:CHEBI:24698',
  'hydroxyflutamide': 'CHEBI:CHEBI:43064',
  'hyperferritinemia': 'HP:HP:0003281',
  'hyperfibrinolysis': 'HP:HP:0040236',
  'hyperfine': 'ungrounded',
  'hyperforin': 'CHEBI:CHEBI:5834',
  'hypertension factor': 'ungrounded',
  'hyphal fragmented': 'ungrounded',
  'hypofractionated': 'ungrounded',
  'lf or a high': 'MESH:D059305'},
 {'MESH:D059305': 'Diet, High-Fat',
  'MESH:D006440': 'Hemofiltration',
  'HGNC:3530': 'F12',
  'MESH:D018859': 'Hair Follicle',
  'CHEBI:CHEBI:5612': 'Halofantrine',
  'MESH:C010176': 'halofuginone',
  'MESH:D006268': 'Health Facilities',
  'MESH:D006333': 'Heart Failure',
  'NCIT:C36169': 'Hemorrhagic Fever',
  'NCIT:C97071': 'Congenital Hepatic Fibrosis',
  'MESH:C548080': 'Neuroferritinopathy',
  'high_frequency': 'high_frequency',
  'high_fructose': 'high_fructose',
  'CHEBI:CHEBI:5721': 'hinokiflavone',
  'MESH:D006620': 'Hip Fractures',
  'MESH:D006624': 'Hippocampus',
  'FPLX:Fibrinogen': 'Fibrinogen',
  'MESH:D005347': 'Fibroblasts',
  'CHEBI:CHEBI:29228': 'hydrogen fluoride',
  'MESH:D015160': 'Hydrops Fetalis',
  'MESH:C431085': 'hydroxyfasudil',
  'CHEBI:CHEBI:24698': 'hydroxyflavone',
  'CHEBI:CHEBI:43064': 'hydroxyflutamide',
  'HP:HP:0003281': 'Increased serum ferritin',
  'HP:HP:0040236': 'Hyperfibrinolysis',
  'CHEBI:CHEBI:5834': 'hyperforin'},
 ['CHEBI:CHEBI:29228',
  'CHEBI:CHEBI:43064',
  'CHEBI:CHEBI:5612',
  'MESH:C010176',
  'MESH:D005347',
  'MESH:D006333',
  'MESH:D006440',
  'MESH:D006620',
  'MESH:D006624',
  'MESH:D018859',
  'MESH:D059305',
  'NCIT:C36169',
  'NCIT:C97071']]

In [7]:
excluded_longforms = []

In [8]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [9]:
additional_entities = {}

In [10]:
unambiguous_agent_texts = {}

In [11]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [15]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [16]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [17]:
intersection1

[('HGNC:17981', 'HGNC:17981', 29)]

In [18]:
intersection2

[('ungrounded', 'HGNC:17981', 0)]

In [20]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=['RTCA', 'RTCD1', 'RPC', 'RTC1', 'RTC'])
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [ ]:
names.update(additional_entitie)

In [12]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-22 05:07:57] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-22 05:10:51] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.9574681352144958 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [13]:
classifier.stats

{'label_distribution': {'MESH:D059305': 876,
  'MESH:D006333': 2740,
  'CHEBI:CHEBI:43064': 23,
  'MESH:D005347': 29,
  'ungrounded': 88,
  'MESH:D006440': 25,
  'CHEBI:CHEBI:29228': 27,
  'HGNC:3530': 7,
  'high_frequency': 213,
  'FPLX:Fibrinogen': 2,
  'CHEBI:CHEBI:5612': 10,
  'MESH:D006624': 17,
  'MESH:D018859': 134,
  'MESH:D006620': 14,
  'high_fructose': 26,
  'MESH:C010176': 56,
  'MESH:C431085': 5,
  'NCIT:C97071': 14,
  'CHEBI:CHEBI:5834': 10,
  'NCIT:C36169': 19,
  'MESH:D006268': 5,
  'MESH:C548080': 3,
  'MESH:D015160': 4,
  'CHEBI:CHEBI:24698': 3,
  'HP:HP:0003281': 3,
  'HP:HP:0040236': 4,
  'CHEBI:CHEBI:5721': 2},
 'f1': {'mean': 0.957468, 'std': 0.008329},
 'precision': {'mean': 0.949712, 'std': 0.009888},
 'recall': {'mean': 0.969886, 'std': 0.006117},
 'MESH:D006440': {'f1': {'mean': 0.686984, 'std': 0.231808},
  'pr': {'mean': 0.6, 'std': 0.252982},
  'rc': {'mean': 0.86, 'std': 0.195959}},
 'high_frequency': {'f1': {'mean': 0.956725, 'std': 0.018552},
  'pr': {'m

In [14]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [15]:
disamb.dump(model_name, results_path)

In [16]:
print(disamb.info())

Disambiguation model for HF

Produces the disambiguations:
	Congenital Hepatic Fibrosis*	NCIT:C97071
	Diet, High-Fat*	MESH:D059305
	F12	HGNC:3530
	Fibrinogen	FPLX:Fibrinogen
	Fibroblasts*	MESH:D005347
	Hair Follicle*	MESH:D018859
	Halofantrine*	CHEBI:CHEBI:5612
	Health Facilities	MESH:D006268
	Heart Failure*	MESH:D006333
	Hemofiltration*	MESH:D006440
	Hemorrhagic Fever*	NCIT:C36169
	Hip Fractures*	MESH:D006620
	Hippocampus*	MESH:D006624
	Hydrops Fetalis	MESH:D015160
	Hyperfibrinolysis	HP:HP:0040236
	Increased serum ferritin	HP:HP:0003281
	Neuroferritinopathy	MESH:C548080
	halofuginone*	MESH:C010176
	high_frequency	high_frequency
	high_fructose	high_fructose
	hinokiflavone	CHEBI:CHEBI:5721
	hydrogen fluoride*	CHEBI:CHEBI:29228
	hydroxyfasudil	MESH:C431085
	hydroxyflavone	CHEBI:CHEBI:24698
	hydroxyflutamide*	CHEBI:CHEBI:43064
	hyperforin	CHEBI:CHEBI:5834

Class level metrics:
--------------------
Grounding                  	Count	F1     
              Heart Failure*	2740	0.97423
        

In [17]:
model_to_s3(disamb)